# Isoforms selection

In [1]:
from Bio import SeqIO
import pandas as pd

In [2]:
isoform_stats = {"gene" : [] ,
                 "transcript" : [] ,
                 "length" : []}

In [3]:
# parsing the input multifasta file
for s in SeqIO.parse("source_data/Theobroma_cacao.Theobroma_cacao_20110822.pep.all.fa", "fasta"):
    isoform_stats['gene'].append(s.description.split(" ")[3])
    isoform_stats['transcript'].append( s.description.split(" ")[4])
    isoform_stats['length'].append(str(len(s.seq)))   

In [4]:
# generating the gene list (without dupicates) for the parsed
# multifasta file 
isoform_stats = pd.DataFrame(isoform_stats)
genes = list(set(isoform_stats['gene']))

In [5]:
# pandas data frame for selected isoforms
selected_isoforms = pd.DataFrame({"gene" : [] ,
                                  "transcript" : [],
                                  "length" : []})
# isoform selection; the longest isoform is selected for
# each gene in "genes"
for i in range(0, len(genes)) :
    gene = genes[i]
    isoforms = isoform_stats.loc[isoform_stats['gene'] == gene]
    isoforms = isoforms.sort_values(by = ['length'], ascending = False)
    selected_isoforms = selected_isoforms.append(isoforms.head(1))

In [6]:
# run to display the array of the selected
# isoforms
selected_isoforms

,gene,transcript,length
7575,gene:TCM_001085,transcript:EOX92068,443
11131,gene:TCM_016844,transcript:EOY02332,350
37357,gene:TCM_014873,transcript:EOY22814,377
7843,gene:TCM_004787,transcript:EOX95237,818
22061,gene:TCM_021857,transcript:EOY07412,299
28120,gene:TCM_008392,transcript:EOX99661,1290
29430,gene:TCM_010145,transcript:EOY00314,228
11216,gene:TCM_019845,transcript:EOY04638,704
22565,gene:TCM_026592,transcript:EOY11405,389
31676,gene:TCM_027112,transcript:EOY25739,165


In [7]:
with open("selected_isoforms.fasta", "w") as out:
    i = 0
    for seq in SeqIO.parse("./source_data/Theobroma_cacao.Theobroma_cacao_20110822.pep.all.fa", "fasta"):
        if to_keep[i].bool :
            SeqIO.write(seq, out, "fasta")
        i += 1

NameError: name 'to_keep' is not defined